# Simulation Assignment - 

**Student:** Suman Budhathoki
**Student Id** 0284292

Simulations and explanations for both parts of the Simulation Assignment: water bottles and plate colors.

## Part 1: 
## Water Bottle Simulation

If there are bottles that usually weigh **10 oz**, but **1 out of 10** bottles is **9 oz**. So, we can explore expectations and simulations.

### 1. Random function for 10 oz (9/10) and 9 oz (1/10)

First we define a function `pull_bottle()` that returns:
- 10 ounces with probability 0.9
- 9 ounces with probability 0.1

In [ ]:
import random

def pull_bottle():
    """Return 10 oz 90% of the time and 9 oz 10% of the time."""
    return 9 if random.random() < 0.1 else 10

# Quick test
[pull_bottle() for _ in range(20)]

### 2. Expected weight of a random pull

The expected value is:

\begin{align*}
E &= 10\times 0.9 + 9 \times 0.1 \\
  &= 9 + 0.9 \\
  &= \mathbf{9.9\text{ ounces}}
\end{align*}

So,the average weight of many random pulls should be **9.9 oz**.

### 3. Simulation of many pulls and comparison



In [ ]:
def simulate_average(n=1000):
    pulls = [pull_bottle() for _ in range(n)]
    return sum(pulls) / n

avg_1000 = simulate_average(1000)
avg_1000

The simulated average above should be **close to 9.9 ounces**, which matches the theoretical expectation.

### 4. Number of pulls required to find the short bottle (9 oz)

The probability of getting a short (9 oz) bottle on any one pull is:  0.1.

The expected number of pulls until the first success in a geometric setting is:

\begin{align*}
E(\text{pulls}) &= \frac{1}{p} = \frac{1}{0.1} = \mathbf{10\ pulls}
\end{align*}



In [ ]:
def pulls_until_short():
    """Simulate pulling bottles until we get a 9 oz bottle.
    Returns the number of pulls required.
    """
    count = 0
    while True:
        count += 1
        if pull_bottle() == 9:
            return count


trials = [pulls_until_short() for _ in range(1000)]

sum(trials) / len(trials)

The simulated shows number of pulls should be **around 10**, which agrees with the theoretical result.

### 5. Sequential search through a fixed case of bottles

Now considering a more realistic model: a single case of 10 bottles where **exactly one** bottle is short (9 oz). We can:

1. Create a list with nine 10-oz bottles and one 9-oz bottle.
2. Shuffle their order.
3. Test bottles in order until we find the 9-oz bottle.

then record how many pulls it took.

In [ ]:
import statistics

def sequential_search():
    """Return the position (1–10) of the 9 oz bottle in a shuffled case."""
    bottles = [10]*9 + [9]  # one short bottle
    random.shuffle(bottles)

    count = 0
    for b in bottles:
        count += 1
        if b == 9:
            return count

# Run several simulations
results = [sequential_search() for _ in range(1000)]
statistics.mean(results), min(results), max(results)

Because the short bottle is equally likely to be in any of the 10 positions, the expected position is the average of the integers 1 through 10:

\begin{align*}
E(\text{position}) &= \frac{1 + 10}{2} = 5.5
\end{align*}

So on average, it takes about **5–6 pulls** to find the 9-oz bottle in this sequential setting. This is **different from** the previous independent model which had expected 10 pulls.

### 6. Improved data structure for real-world use

In practice, bottles may be stored in cases with labels, IDs, and more information. A more realistic data structure can:

- Track each bottle's **ID** and **weight**.
- Store whether a bottle has already been **tested**.
- Easily extend to multiple cases, locations, etc.

Below is a simple class-based design that could be expanded to a real-world system.

In [ ]:
class BottleCase:
    def __init__(self, case_id="Case-1", n=10):
        self.case_id = case_id
        self.bottles = [{"id": f"{case_id}-B{i}", "weight": 10, "tested": False}
                        for i in range(n)]
        # randomly choose one bottle to be short
        short_index = random.randrange(n)
        self.bottles[short_index]["weight"] = 9

    def pull_and_test(self, index):
        """Test a specific bottle by index and mark it as tested."""
        bottle = self.bottles[index]
        bottle["tested"] = True
        return bottle

    def random_test(self):
        """Randomly pick a not-yet-tested bottle and test it."""
        untested = [b for b in self.bottles if not b["tested"]]
        bottle = random.choice(untested)
        bottle["tested"] = True
        return bottle

# Example usage
case = BottleCase()

# Randomly test bottles until we find the 9 oz one
count = 0
for _ in range(len(case.bottles)):
    count += 1
    b = case.random_test()
    if b["weight"] == 9:
        print(f"Found short bottle after {count} tests: {b}")
        break

This object-oriented structure is better suited for a real-world application where each bottle and case may need to be tracked, logged, or stored in a database.

## Part 2: Simulating Plates

now in model a stack of plates in different colors:

- Total plates: **24**  
- Colors: pink, blue, black, green  
- Each color has **6 plates**.

We pay special attention to pink plates because the daughter will only eat from a pink plate.

In [ ]:
# Base list of plates
plates = ["pink"]*6 + ["blue"]*6 + ["green"]*6 + ["black"]*6
len(plates), {c: plates.count(c) for c in set(plates)}

### 1. Probability of drawing a pink plate

There are 6 pink plates out of 24 total plates:

\begin{align*}
P(\text{pink}) &= \frac{6}{24} = \frac{1}{4} = 0.25
\end{align*}

So the probability of getting a pink plate on a single random draw is **0.25 (25%)**.

### 2. Probability that in four plates for dinner, at least one is pink

We can use the complement rule:

1. First compute the probability that **none** of the four plates is pink.
2. Then subtract from 1.

\begin{align*}
P(\text{not pink}) &= 1 - 0.25 = 0.75 \\
P(\text{no pink in 4 draws}) &= 0.75^4 \approx 0.316 \\
P(\text{at least one pink}) &= 1 - 0.75^4 \approx 1 - 0.316 = 0.684
\end{align*}

So the probability that at least one of the four plates is pink is approximately **0.684 (68.4%)**.

### 3. Randomization test of the above theoretical probabilities

Estimatating these probabilities using simulation.

- Probability of a single pink draw.
- Probability that at least one of four plates is pink.

In [ ]:
def draw_one_plate():
    return random.choice(plates)

def draw_four_plates_no_replacement():
    return random.sample(plates, 4)

def simulate_single_pink(trials=5000):
    count_pink = 0
    for _ in range(trials):
        if draw_one_plate() == "pink":
            count_pink += 1
    return count_pink / trials

def simulate_at_least_one_pink(trials=5000):
    count_at_least_one = 0
    for _ in range(trials):
        sample = draw_four_plates_no_replacement()
        if "pink" in sample:
            count_at_least_one += 1
    return count_at_least_one / trials

sim_single = simulate_single_pink()
sim_four = simulate_at_least_one_pink()

sim_single, sim_four

The simulation results should be close to:

- `sim_single` ≈ **0.25**  
- `sim_four` ≈ **0.684**  

which matches the theoretical calculations.

### Helper: Creating a plate stack

Treating a stack as a list where the **last element** (`stack[-1]`) is the **top** of the stack.

In [ ]:
def make_stack():
    stack = plates.copy()
    random.shuffle(stack)
    return stack

# Example stack
make_stack()[:10]

### 4. Probability the top plate is pink (stack model)

Even when the plates are in a stack, as long as all orders are equally likely, the chance that the **top** plate is pink is still:

\[ P(\text{top is pink}) = \frac{6}{24} = 0.25. \]

We can verify this with simulation.

In [ ]:
def prob_pink_top(trials=5000):
    count_pink_top = 0
    for _ in range(trials):
        stack = make_stack()
        if stack[-1] == "pink":  # top of stack
            count_pink_top += 1
    return count_pink_top / trials

prob_pink_top()

The result should again be close to **0.25**, matching our theoretical probability.

### 5. Dishwashing behavior: top 12 plates shuffled and returned

Now model the way plates are washed:

- About 12 plates are used and then placed in the dishwasher.
- After washing, these 12 plates are shuffled and placed **on top** of the remaining stack.

This may change the distribution of colors near the top of the stack over time.

In [ ]:
def reshuffle_top_12(stack):
    """Remove the top 12 plates, shuffle them, and put them back on top."""
    if len(stack) <= 12:
        # if fewer than or equal to 12, just shuffle all
        random.shuffle(stack)
        return stack

    top12 = stack[-12:]
    bottom = stack[:-12]
    random.shuffle(top12)
    return bottom + top12

# Quick demonstration
s = make_stack()
reshuffle_top_12(s)[:10]

### 6. Daughter's behavior: rejecting non-pink plates

The daughter will only eat off a pink plate. If the top plate is not pink:

1. She rejects that plate; it is temporarily removed.
2. Another plate becomes the new top; she repeats the process.
3. When she finally gets a pink plate, the rejected plates are later returned to the stack (e.g., when washing dishes).

Below is one way to model **a single meal** where she keeps rejecting plates until she gets pink. We also return the rejected plates to random positions in the stack afterwards to mimic plates being washed and reinserted.

In [ ]:
def daughter_pick(stack):
    """Simulate the daughter choosing a plate from the stack.
    She keeps rejecting plates until a pink one appears on top.
    The rejected plates are later returned to random positions.
    Returns the color of the plate she finally uses.
    """
    rejected = []

    # Ensure there is at least one pink plate somewhere
    if "pink" not in stack:
        raise ValueError("No pink plates in the stack!")

    # Keep popping from the top until we see a pink plate
    while stack[-1] != "pink":
        rejected.append(stack.pop())

    # She uses the pink plate
    pink_plate = stack.pop()

    # Return the rejected plates at random positions in the stack
    for plate in rejected:
        pos = random.randint(0, len(stack))
        stack.insert(pos, plate)

    return pink_plate

# Quick test
st = make_stack()
reshuffle_top_12(st)
daughter_pick(st)

### 7. Long-run behavior over many days

We can now model **1000 dinners**. Each day:

1. The plates used previously might have been washed and mixed into the top (using `reshuffle_top_12`).
2. The daughter chooses her plate using `daughter_pick`.

We can now track how often she ends up with a pink plate.

In [ ]:
def run_experiment(days=1000):
    stack = make_stack()
    pink_count = 0

    for _ in range(days):
        # simulate dishwashing effect
        stack = reshuffle_top_12(stack)

        # daughter chooses her plate
        chosen = daughter_pick(stack)
        if chosen == "pink":
            pink_count += 1

        # used plate gets washed and eventually re-added
        # we reinsert the used plate at a random position in the stack
        pos = random.randint(0, len(stack))
        stack.insert(pos, chosen)

    return pink_count / days

run_experiment()

The result should be very close to **1.0**, indicating that the daughter almost always ends up with a pink plate due to repeatedly rejecting other colors until a pink one appears.

### 8. Usage count for each plate color

Finally, we can track **how often each color is used** over many meals. This gives insight into whether the pink plates are used disproportionately compared to the others.

In [ ]:
from collections import Counter

def usage_counts(days=1000):
    stack = make_stack()
    counts = Counter()

    for _ in range(days):
        stack = reshuffle_top_12(stack)
        chosen = daughter_pick(stack)
        counts[chosen] += 1
        # add used plate back somewhere in the stack
        pos = random.randint(0, len(stack))
        stack.insert(pos, chosen)

    return counts

usage_counts(1000)

We expect the **pink plates** to be used significantly more often than the other colors, reflecting the daughter's preference and rejection behavior.

----------------------

The End
Suman Budhathoki
0284292